In [1]:
# Kaggle 데이터 파일의 앞부분 3줄만 읽어서 확인하는 코드
path_kaggle = '학습용/poi_review_corpus.txt' 

print("--- [Kaggle 파일 내용 확인] ---")
with open(path_kaggle, 'r', encoding='utf-8') as f:
    for i in range(5): # 5줄만 출력
        print(f.readline().strip())

--- [Kaggle 파일 내용 확인] ---
id	content	label
139363	한번 가 봤는데 떡 식감이 너무 안좋았음. 떡볶이 쏘스도 별로였음. 단 참치 주먹밥은 먹을만 했음. 그 외에는 그냥 그냥 함	0
155416	싸고 맛있고 최고 분짜도 갠춘 다만 팟타이는 별로	1
148176	가격 감안해도 참 맛없음	0
143040	직원들 불친절함 바쁜건가?	0


In [2]:
# 파일 경로 (본인 경로에 맞게 수정!)
path_nsmc = '학습용/ratings_train.txt'

print("--- [NSMC 파일 내용 확인] ---")
try:
    with open(path_nsmc, 'r', encoding='utf-8') as f:
        for i in range(5): # 5줄만 출력
            print(f.readline().strip())
except Exception as e:
    print(f"파일을 읽을 수 없습니다: {e}")

--- [NSMC 파일 내용 확인] ---
id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0


In [3]:
import pandas as pd

# ==========================================
# 1. 파일 경로 설정 (본인 경로가 맞는지 마지막 확인!)
# ==========================================
path_nsmc = '학습용/ratings_train.txt'       
path_kaggle = '학습용/poi_review_corpus.txt'     
path_aihub = '학습용/aihub_shuffled_15k.csv' 

# ==========================================
# 2. 데이터 로드 및 병합
# ==========================================
combined_data = []

# (1) NSMC 데이터 처리
print("1️⃣ NSMC(영화) 데이터 로드 중...")
try:
    # 탭으로 구분, 첫 줄 헤더 있음 -> read_table 사용하면 자동 처리됨
    df_nsmc = pd.read_table(path_nsmc)
    df_nsmc = df_nsmc[['document', 'label']] 
    df_nsmc.columns = ['text', 'label']      
    df_nsmc.dropna(inplace=True)             
    combined_data.append(df_nsmc)
    print(f"   ✅ NSMC 완료: {len(df_nsmc)}개")
except Exception as e:
    print(f"   ❌ NSMC 로드 실패: {e}")

# (2) Kaggle POI 데이터 처리
print("2️⃣ Kaggle(장소) 데이터 로드 중...")
try:
    # 탭으로 구분(\t) 확인 완료
    df_kaggle = pd.read_csv(path_kaggle, sep='\t')
    
    # 컬럼: id, content, label -> 필요한 건 content, label
    df_kaggle = df_kaggle[['content', 'label']]
    df_kaggle.columns = ['text', 'label'] # 이름 통일
    
    df_kaggle.dropna(inplace=True)
    combined_data.append(df_kaggle)
    print(f"   ✅ Kaggle 완료: {len(df_kaggle)}개")
except Exception as e:
    print(f"   ❌ Kaggle 로드 실패: {e}")

# (3) AI Hub 데이터 처리
print("3️⃣ AI Hub(쇼핑) 데이터 로드 중...")
try:
    # 이미 전처리된 CSV 파일
    df_aihub = pd.read_csv(path_aihub)
    df_aihub = df_aihub[['text', 'label']]
    combined_data.append(df_aihub)
    print(f"   ✅ AI Hub 완료: {len(df_aihub)}개")
except Exception as e:
    print(f"   ❌ AI Hub 로드 실패: {e}")

# ==========================================
# 3. 최종 병합 및 저장
# ==========================================
if combined_data:
    final_df = pd.concat(combined_data, ignore_index=True)
    
    # 데이터 섞기 (Shuffle) - 학습 시 매우 중요!
    final_df = final_df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print("\n" + "="*30)
    print(f"🎉 최종 통합 완료! 총 데이터 개수: {len(final_df)}개")
    print("="*30)
    print("[라벨 분포 확인]")
    print(final_df['label'].value_counts()) 
    
    # 최종 저장
    final_df.to_csv("final_train_data.csv", index=False, encoding='utf-8-sig')
    print("\n💾 'final_train_data.csv' 파일 생성이 완료되었습니다.")
else:
    print("⚠️ 병합할 데이터가 없습니다. 경로를 확인해주세요.")

1️⃣ NSMC(영화) 데이터 로드 중...
   ✅ NSMC 완료: 149995개
2️⃣ Kaggle(장소) 데이터 로드 중...
   ✅ Kaggle 완료: 100000개
3️⃣ AI Hub(쇼핑) 데이터 로드 중...
   ✅ AI Hub 완료: 15023개

🎉 최종 통합 완료! 총 데이터 개수: 265018개
[라벨 분포 확인]
label
1    137182
0    127836
Name: count, dtype: int64

💾 'final_train_data.csv' 파일 생성이 완료되었습니다.
